In [ ]:
# 라이브러리 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings(action='ignore')
%config InlineBackend.figure_format='retina'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = '/content/drive/MyDrive/spaceship_titanic/'
train = pd.read_csv(path+'train.csv')
test = pd.read_csv(path+'test.csv')

In [ ]:
train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [ ]:
train.describe()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,8514.000000,8512.000000,8510.000000,8485.000000,8510.000000,8505.000000
mean,28.827930,224.687617,458.077203,173.729169,311.138778,304.854791
std,14.489021,666.717663,1611.489240,604.696458,1136.705535,1145.717189
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,38.000000,47.000000,76.000000,27.000000,59.000000,46.000000
max,79.000000,14327.000000,29813.000000,23492.000000,22408.000000,24133.000000


In [ ]:
train.isna().sum()

,0
PassengerId,0
HomePlanet,201
CryoSleep,217
Cabin,199
Destination,182
Age,179
VIP,203
RoomService,181
FoodCourt,183
ShoppingMall,208


In [ ]:
train['Transported'].value_counts()

,count
Transported,
True,4378
False,4315


In [ ]:
def split_cabin(row):
    if pd.isna(row):
        return pd.Series([np.nan, np.nan, np.nan])
    parts = row.split('/')
    return pd.Series(parts)

train[['Deck', 'RoomNum', 'Side']] = train['Cabin'].apply(split_cabin)
test[['Deck', 'RoomNum', 'Side']] = test['Cabin'].apply(split_cabin)

In [ ]:
train.drop(columns=['Name','Cabin'],inplace=True)
test.drop(columns=['Name','Cabin'],inplace=True)

In [ ]:
cat_cols = ['HomePlanet','Destination','CryoSleep','VIP','Deck','Side']
for col in cat_cols:
    train[col].fillna("Unknown", inplace=True)
    test[col].fillna("Unknown", inplace=True)

In [ ]:
train['RoomNum'] = pd.to_numeric(train['RoomNum'], errors='coerce')
test['RoomNum'] = pd.to_numeric(test['RoomNum'], errors='coerce')

In [ ]:
num_cols = ['Age','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck','RoomNum']
for col in num_cols:
    median_value = train[col].median()
    train[col].fillna(median_value, inplace=True)
    test[col].fillna(median_value, inplace=True)

In [ ]:
train.isna().sum()

,0
PassengerId,0
HomePlanet,0
CryoSleep,0
Destination,0
Age,0
VIP,0
RoomService,0
FoodCourt,0
ShoppingMall,0
Spa,0


In [ ]:
# 1) 테스트용 PassengerId 저장 (제출용)
test_id = test['PassengerId']

# 2) 학습(X)에서는 PassengerId 제거
train = train.drop('PassengerId', axis=1)

In [ ]:
test = test.drop('PassengerId', axis=1)

In [ ]:
# Target 설정
target = ['Transported']

# 데이터 분리
X = train.drop(target, axis=1)
y = train.loc[:, target]

In [ ]:
y = y.astype(int)

In [ ]:
dumm_cols = ['HomePlanet','CryoSleep','Destination', 'VIP','Side','Deck']

X = pd.get_dummies(X,columns=dumm_cols,  dtype=int)
test = pd.get_dummies(test, columns=dumm_cols, dtype=int)

In [ ]:
# 모듈 불러오기
from sklearn.model_selection import train_test_split

# 7:3으로 분리
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state=1)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

from sklearn.metrics import *

In [ ]:
model1 = RandomForestClassifier(n_estimators=300,
                                max_depth=15,
                                random_state=42,
                                n_jobs=1)

In [ ]:
model1.fit(X_train, y_train)

RandomForestClassifier(max_depth=15, n_estimators=300, n_jobs=1,
                       random_state=42)

In [ ]:
y_pred = model1.predict(X_val)

In [ ]:
print(confusion_matrix(y_val,y_pred))
print(classification_report(y_val,y_pred))


[[709 134]
 [198 698]]
              precision    recall  f1-score   support

           0       0.78      0.84      0.81       843
           1       0.84      0.78      0.81       896

    accuracy                           0.81      1739
   macro avg       0.81      0.81      0.81      1739
weighted avg       0.81      0.81      0.81      1739



In [ ]:
model2 = LGBMClassifier(
    n_estimators=500,
    learning_rate=0.05,
    random_state=42
)

In [ ]:
model2.fit(X_train, y_train)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 3482, number of negative: 3472
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001700 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1657
[LightGBM] [Info] Number of data points in the train set: 6954, number of used features: 32
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500719 -> initscore=0.002876
[LightGBM] [Info] Start training from score 0.002876


LGBMClassifier(learning_rate=0.05, n_estimators=500, random_state=42)

In [ ]:
y_pred2 = model2.predict(X_val)

In [ ]:
print(confusion_matrix(y_val,y_pred2))
print(classification_report(y_val,y_pred2))

[[698 145]
 [177 719]]
              precision    recall  f1-score   support

           0       0.80      0.83      0.81       843
           1       0.83      0.80      0.82       896

    accuracy                           0.81      1739
   macro avg       0.81      0.82      0.81      1739
weighted avg       0.82      0.81      0.81      1739



In [ ]:
model3 = XGBClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    eval_metric='logloss'
)

In [ ]:
model3.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=500, n_jobs=None,
              num_parallel_tree=None, ...)

In [ ]:
y_pred3 = model3.predict(X_val)

In [ ]:
print(confusion_matrix(y_val,y_pred3))
print(classification_report(y_val,y_pred3))

[[697 146]
 [164 732]]
              precision    recall  f1-score   support

           0       0.81      0.83      0.82       843
           1       0.83      0.82      0.83       896

    accuracy                           0.82      1739
   macro avg       0.82      0.82      0.82      1739
weighted avg       0.82      0.82      0.82      1739



In [ ]:
best_model = XGBClassifier(
    n_estimators=1000,
    learning_rate=0.03,
    max_depth=7,
    min_child_weight=3,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1,
    eval_metric='logloss'
)

best_model.fit(X, y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.03, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=7, max_leaves=None,
              min_child_weight=3, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1000, n_jobs=-1,
              num_parallel_tree=None, ...)

In [ ]:
test_pred = best_model.predict(test)

In [ ]:
submission = pd.DataFrame({
    'PassengerId': test_id,
    'Transported': test_pred.astype(bool)
})

submission.to_csv('submission_xgb.csv', index=False)

In [ ]:
from google.colab import files
files.download('submission_xgb.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>